In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/D_LABITEMS/D_LABITEMS.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/CALLOUT/CALLOUT_sorted.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/CALLOUT/CALLOUT_random.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/DATETIMEEVENTS/DATETIMEEVENTS_CV_sorted.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/DATETIMEEVENTS/DATETIMEEVENTS_CV_random.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/ICUSTAYS/ICUSTAYS_sorted.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/ICUSTAYS/ICUSTAYS_random.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/MICROBIOLOGYEVENTS/MICROBIOLOGYEVENTS_random.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/MICROBIOLOGYEVENTS/MICROBIOLOGYEVENTS_sorted.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/CPTEVENTS/CPTEVENTS_sorted.csv
/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)/CPTEVENTS/CPTEVENTS_random.cs

In [2]:
# File paths
base_path = "/kaggle/input/mimic-iii-10k/MIMIC -III (10000 patients)"

# Load CSVs
admissions = pd.read_csv(f"{base_path}/ADMISSIONS/ADMISSIONS_sorted.csv")
patients = pd.read_csv(f"{base_path}/PATIENTS/PATIENTS_sorted.csv")
diagnoses = pd.read_csv(f"{base_path}/DIAGNOSES_ICD/DIAGNOSES_ICD_sorted.csv")

In [3]:
print("Admissions:")
display(admissions.head())

print("\n Admissions Info:\n")
display(admissions.info())

Admissions:


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1



 Admissions Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12911 entries, 0 to 12910
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                12911 non-null  int64 
 1   SUBJECT_ID            12911 non-null  int64 
 2   HADM_ID               12911 non-null  int64 
 3   ADMITTIME             12911 non-null  object
 4   DISCHTIME             12911 non-null  object
 5   DEATHTIME             1287 non-null   object
 6   ADMISSION_TYPE        12911 non-null  object
 7   ADMISSION_LOCATION    12911 non-null  object
 8   DISCHARGE_LOCATION    12911 non-null  object
 9   INSURANCE             12911 non-null  object
 10  LANGUAGE              3445 non-null   object
 11  RELIGION              12728 non-null  object
 12  MARITAL_STATUS        9714 non-null   object
 13  ETHNICITY             12911 non-null  object
 14  EDREGTIME             5932 non-null   object
 15  EDOUTTIME       

None

### Understanding the Admissions Table
* `SUBJECT_ID` : Unique patient identifier
* `HADM_ID` : Unique hospital admission ID
* `ADMITTIME`, `DISHTIME` : Used to calculate length of stay and time between visits
* `DEATHTIME` : May indicate patients who died during or shortly after admission
* `ADMISSION_TYPE` : E.g., EMERGENCY, NEWBORN
* `HOSPITAL_EXPIRE_FLAG` : 1 = patient died during hospital stay
* **Target Construction :** If a patient has another `ADMITTIME` within 30 days of `DISCHTIME`, that admission is flagged as **readmitted**

In [4]:
print("Patients:")
display(patients.head())

print("\n Patients Info:\n")
display(patients.info())

Patients:


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,1,2,M,2138-07-17 00:00:00,NaN,NaN,NaN,0
1,2,3,M,2025-04-11 00:00:00,2102-06-14 00:00:00,NaN,2102-06-14 00:00:00,1
2,3,4,F,2143-05-12 00:00:00,NaN,NaN,NaN,0
3,4,5,M,2103-02-02 00:00:00,NaN,NaN,NaN,0
4,5,6,F,2109-06-21 00:00:00,NaN,NaN,NaN,0



 Patients Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ROW_ID       10000 non-null  int64 
 1   SUBJECT_ID   10000 non-null  int64 
 2   GENDER       10000 non-null  object
 3   DOB          10000 non-null  object
 4   DOD          3761 non-null   object
 5   DOD_HOSP     2192 non-null   object
 6   DOD_SSN      3438 non-null   object
 7   EXPIRE_FLAG  10000 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 625.1+ KB


None

### Understanding the Patients Table
* `GENDER`, `DOB` : For calculating age at admission
* `EXPIRE_FLAG`, `DOD_HOSP`, `DOD` : Useful for removing deceased from modeling or for labeling outcomes

In [5]:
print("Diagnoses:")
display(diagnoses.head())

print("\n Diagnoses Info:\n")
display(diagnoses.info())

Diagnoses:


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1,2,163353,1.0,V3001
1,2,2,163353,2.0,V053
2,3,2,163353,3.0,V290
3,12,3,145834,9.0,2639
4,10,3,145834,7.0,6826



 Diagnoses Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118300 entries, 0 to 118299
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      118300 non-null  int64  
 1   SUBJECT_ID  118300 non-null  int64  
 2   HADM_ID     118300 non-null  int64  
 3   SEQ_NUM     118287 non-null  float64
 4   ICD9_CODE   118287 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 4.5+ MB


None

### Understanding the Diagnoses Table
* `ICD9_CODE` : Can be grouped to predict readmission risk

## Create Readmission Labels

In [6]:
from datetime import timedelta

# convert to datetime
admissions["ADMITTIME"] = pd.to_datetime(admissions["ADMITTIME"])
admissions["DISCHTIME"] = pd.to_datetime(admissions["DISCHTIME"])

In [7]:
# sort by patient and admission time
admissions = admissions.sort_values(by=["SUBJECT_ID", "ADMITTIME"]).reset_index(drop=True)

In [9]:
# creating an admission label - by default 0 : not readmitted
admissions["READMISSION_LABEL"] = 0

# loop through each patient
for i in range(len(admissions) - 1):
    current = admissions.iloc[i]
    next_row = admissions.iloc[i + 1]

    # check if same patient and next admission is within 30 days
    if current["SUBJECT_ID"] == next_row["SUBJECT_ID"]:
        days_between = (next_row["ADMITTIME"] - current["DISCHTIME"]).days

        if 0 < days_between <= 30:
            admissions.at[i, "READMISSION_LABEL"] = 1

In [16]:
admissions["READMISSION_LABEL"].value_counts()

READMISSION_LABEL
0    94.276199
1     5.723801
Name: proportion, dtype: float64